# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# load in csv
csvpath = "cities.csv"

# read in csv as dataframe
vacation_df = pd.read_csv(csvpath)
vacation_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Aykhal,66.00,111.50,51.80,87,90,4.47,RU,1595102745
1,Qiyang,26.59,111.85,80.92,76,100,6.60,CN,1595102745
2,Hilo,19.73,-155.09,84.20,62,75,4.70,US,1595102514
3,Leningradskiy,69.38,178.42,37.72,82,100,10.38,RU,1595102745
4,Albany,42.60,-73.97,91.99,36,3,4.79,US,1595102607


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure gmaps
gmaps.configure(api_key=g_key)

locations = vacation_df[["Lat", "Lng"]].astype(float)
humidity = vacation_df["Humidity"].astype(float)

In [4]:
# set center and zoom levels
center = (10, 0)
fig = gmaps.figure(center=center, zoom_level=2.2)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=500,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# narrow down cities to fit weather conditions
ideal_weather = vacation_df[(vacation_df["Max Temp"]>= 80)&
                            (vacation_df["Max Temp"]<=85)&
                            (vacation_df["Wind Speed"]<=10)&
                            (vacation_df["Cloudiness"]<=10)]

# drop any rows with null values
ideal_weather = ideal_weather.dropna()
ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
140,Ammon,43.47,-111.97,81.00,41,1,6.93,US,1595102155
244,Rāvar,31.27,56.81,83.01,15,0,6.06,IR,1595102373
309,Umluj,25.02,37.27,83.68,63,0,5.95,SA,1595102382
341,Erzin,36.96,36.20,80.01,79,1,0.78,TR,1595102386
475,Morro Bay,35.37,-120.85,84.00,60,1,7.02,US,1595102152
538,Puerto del Rosario,28.50,-13.86,84.20,58,0,6.93,ES,1595102423


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame(ideal_weather)
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
140,Ammon,43.47,-111.97,81.00,41,1,6.93,US,1595102155,
244,Rāvar,31.27,56.81,83.01,15,0,6.06,IR,1595102373,
309,Umluj,25.02,37.27,83.68,63,0,5.95,SA,1595102382,
341,Erzin,36.96,36.20,80.01,79,1,0.78,TR,1595102386,
475,Morro Bay,35.37,-120.85,84.00,60,1,7.02,US,1595102152,


In [7]:
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# iterate through dataframe
for i, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using base url and params dictionary
    response = requests.get(base_url, params=params)
    response = response.json()
    try:
        hotel_df.loc[i, "Hotel Name"] = response["results"][0]["name"]
    except (KeyError, IndexError):
        pass
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
140,Ammon,43.47,-111.97,81.00,41,1,6.93,US,1595102155,Holiday Inn Express & Suites Idaho Falls
244,Rāvar,31.27,56.81,83.01,15,0,6.06,IR,1595102373,computertik
309,Umluj,25.02,37.27,83.68,63,0,5.95,SA,1595102382,Moon light Furnished Units
341,Erzin,36.96,36.20,80.01,79,1,0.78,TR,1595102386,Artemis Otel
475,Morro Bay,35.37,-120.85,84.00,60,1,7.02,US,1595102152,Inn at Morro Bay
538,Puerto del Rosario,28.50,-13.86,84.20,58,0,6.93,ES,1595102423,Hotel JM Puerto del Rosario


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))